In [26]:
from esa_snappy import ProductIO, HashMap, GPF, ProductUtils,jpy
import os,gc
import geopandas as gpd
from shapely import wkt
import pickle
import sys
import json
gc.enable()
gc.collect()

0

In [27]:
json_dt='/data/ksa/01_Image_Acquisition/05_Json_Coverage/32_coverage_ASF.json'
with open(json_dt,'r') as file:
    json_file=json.load(file)

In [28]:
#[wkt.load()json_file[0]['geometry']]#
total_bounds=gpd.GeoDataFrame(geometry=[wkt.loads(json_file[0]['geometry'])]).total_bounds
north=total_bounds[0]
south=total_bounds[2]
west=total_bounds[1]
east=total_bounds[3]

In [29]:
total_bounds

array([105.90265807,  -6.33207401, 106.80800513,  -5.42552994])

In [30]:
def do_subset(source, wkt):
    parameters = HashMap()
    parameters.put('geoRegion', wkt)
    output = GPF.createProduct('Subset', parameters, source)
    return output
def do_reproject(source):
    parameters = HashMap()
    parameters.put('crs', 'EPSG:4326')  # Replace with your target CRS
    #parameters.put('resolution', 'highest')
    output = GPF.createProduct('Reproject', parameters, source)
    return output

In [24]:
def run_subseting(wkt,list_sources):
    for i in range(len(list_sources)):
        print(list_sources[i])
        p=ProductIO.readProduct('/data/ksa/01_Image_Acquisition/02_Processed_Image/'+list_sources[i]+'.tif')
        reproject=do_reproject(p)
        subset=do_subset(reproject,wkt)
        ProductIO.writeProduct(subset, 'REP1_'+list_sources[i], 'GeoTIFF')
        p.dispose()
        p.closeIO()
        del subset
        del reproject

In [25]:
run_subseting(json_file[0]['geometry'],[json_file[0]['periode'][0]['image_asf'][0]])

S1A_IW_GRDH_1SDV_20210108T111521_20210108T111546_036045_043961_718B

100% done.

100% done.


Warning 1: /data/ksa/01_Image_Acquisition/02_Processed_Image/S1A_IW_GRDH_1SDV_20210108T111521_20210108T111546_036045_043961_718B.tif: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: /data/ksa/01_Image_Acquisition/02_Processed_Image/S1A_IW_GRDH_1SDV_20210108T111521_20210108T111546_036045_043961_718B.tif: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: /data/ksa/01_Image_Acquisition/02_Processed_Image/S1A_IW_GRDH_1SDV_20210108T111521_20210108T111546_036045_043961_718B.tif: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. 

In [9]:
def mosaicing(list_sources,bound,outflnm):
    #print('Mosaicking Process Begin')
    products = jpy.array('org.esa.snap.core.datamodel.Product',len(list_sources))
    Variable = jpy.get_type('org.esa.snap.core.gpf.common.MosaicOp$Variable')
    variables = jpy.array('org.esa.snap.core.gpf.common.MosaicOp$Variable', 2)
    
    parameters = HashMap()
    parameters.put('combine', 'OR')
    parameters.put('crs', 'EPSG:3857')
    parameters.put('resampling', 'Nearest')
    parameters.put('pixelSizeX', 20.0)
    parameters.put('pixelSizeY', 20.0)
    parameters.put('orthorectify', False)
    parameters.put('westBound',bound[0])
    parameters.put('southBound',bound[1])
    parameters.put('northBound',bound[3])
    parameters.put('eastBound',bound[2])
    
    for i in range(len(list_sources)):
        p=ProductIO.readProduct('/data/ksa/01_Image_Acquisition/02_Processed_Image/'+list_sources[i]+'.tif')
        products[i]=p
    if len(list_sources)>0:
        band_names = products[0].getBandNames()
        i=0
        for band in band_names:
            print(band)
            variables[i]=Variable(band,band)
            i=i+1 
    parameters.put('variables', variables) 
    output = GPF.createProduct('Mosaic', parameters, products)
    ProductIO.writeProduct(output, outflnm, 'GeoTIFF')
    for i in range(len(list_sources)):
        products[i].dispose()
        products[i].closeIO()

In [10]:
mosaicing(json_file[0]['periode'][0]['image_asf'],
         total_bounds,'temp')

Warning 1: /data/ksa/01_Image_Acquisition/02_Processed_Image/S1A_IW_GRDH_1SDV_20210108T111521_20210108T111546_036045_043961_718B.tif: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: /data/ksa/01_Image_Acquisition/02_Processed_Image/S1A_IW_GRDH_1SDV_20210104T223357_20210104T223424_035994_043783_678D.tif: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: /data/ksa/01_Image_Acquisition/02_Processed_Image/S1A_IW_GRDH_1SDV_

band_1
band_2

100% done.

100% done.

100% done.

100% done.


Warning 1: /data/ksa/01_Image_Acquisition/02_Processed_Image/S1A_IW_GRDH_1SDV_20210108T111521_20210108T111546_036045_043961_718B.tif: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: /data/ksa/01_Image_Acquisition/02_Processed_Image/S1A_IW_GRDH_1SDV_20210108T111521_20210108T111546_036045_043961_718B.tif: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: /data/ksa/01_Image_Acquisition/02_Processed_Image/S1A_IW_GRDH_1SDV_20210104T223332_20210104T223357_035994_043783_3182.tif: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: /data/ksa/01_Image_Acquisition/02_Processed_Image/S1A_IW_GRDH_1SDV_20210108T111521_20210108T111546_036045_043961_

In [ ]:

def mosaicing(list_sources,bound,outflnm,band):
    #print('Mosaicking Process Begin')
    products = jpy.array('org.esa.snap.core.datamodel.Product',len(list_sources))
    Variable = jpy.get_type('org.esa.snap.core.gpf.common.MosaicOp$Variable')
    variables = jpy.array('org.esa.snap.core.gpf.common.MosaicOp$Variable', 1)
    
    parameters = HashMap()
    parameters.put('combine', 'OR')
    parameters.put('crs', 'EPSG:3857')
    parameters.put('resampling', 'Nearest')
    parameters.put('pixelSizeX', 20.0)
    parameters.put('pixelSizeY', 20.0)
    parameters.put('orthorectify', False)
    parameters.put('westBound',bound[0])
    parameters.put('southBound',bound[1])
    parameters.put('northBound',bound[2])
    parameters.put('eastBound',bound[3])
    str_='Gamma0_'+band+'_db'
    variables[0]=Variable(str_,str_)
    parameters.put('variables', variables) 
    #print('Data:')
    for i in range(len(list_sources)):
    #    print(list_sources[i])
        p=ProductIO.readProduct(list_sources[i])
        products[i]=p
    output = GPF.createProduct('Mosaic', parameters, products)
    #print('Done')
    ProductIO.writeProduct(output, outflnm, 'GeoTIFF')
    #print('+++++++++++++++++++++++++++++++++++++++++')
    for i in range(len(list_sources)):
    #    print(list_sources[i])
        products[i]=None

def running_mosaicing(str_pickle,path_shp):
    gdf_=gpd.read_file(path_shp)
    with open(str_pickle,'rb') as file:
        dict_grid_date=pickle.load(file)
    for grid in dict_grid_date.keys():
        boundary=gdf_.query('ID_GRID==@grid')
        west=boundary['left'].values[0]
        east=boundary['right'].values[0]
        north=boundary['top'].values[0]
        south=boundary['bottom'].values[0]
        bound=[west,south,north,east]
        for bands in ['VH','VV']:
            for period_range in dict_grid_date[grid][bands].keys():
                lst_sources=dict_grid_date[grid][bands][period_range]
                path_sources=[]
                if len(lst_sources)>0:
                    for i in lst_sources:
                        path_sources.append('shared_ksa/00_SATELLITE IMAGERY/01_PREPROCESSING_1/'+grid+'/'+bands+'/'+i)
                    outflnm='shared_ksa/00_SATELLITE IMAGERY/01_MOSAIC/'+grid+'/'+bands+'/'+period_range
                    if os.path.exists(outflnm+'.tif'):
                        print(outflnm,' is already exists')
                        pass
                    else:
                        print('running for ', outflnm)
                        mosaicing(path_sources,bound,outflnm,bands)
        


In [ ]:

if __name__=="__main__":
    str_pickle=sys.argv[1]
    path_shp=sys.argv[2]
    running_mosaicing(str_pickle,path_shp)